In [5]:
import numpy as np
from data_handler import StaticDataHandler
from model_builder import ModelBuilder
from constants import *
data = StaticDataHandler._load_h5py_data(H5PY_PATH)
print(len(data['targets']))
print(len(data['inputs']))

print("Train Test")
_train_data, _test_data = StaticDataHandler._train_split_test(split_ratio=0.2, input_data=data)
print("Train Validation")
_train_data, _validation_data = StaticDataHandler._train_split_test(split_ratio=0.2, input_data=_train_data)
mappings = StaticDataHandler._load_jsonfile_into_song(MAPPING_PATH)

Datasets in the file: ['inputs', 'targets']
Inputs shape: (1729818, 64, 46)
Targets shape: (1729818,)
1729818
1729818
Train Test
Train Validation


In [7]:
print("Data type:", type(mappings))
print("Total length:", len(mappings))
print("Inputs Train Length", len(_train_data["inputs"]))
print("Inputs Validation Length", len(_validation_data["inputs"]))
print("Inputs Test Length", len(_test_data["inputs"]))
print("Total Length: ", len(_train_data["inputs"]) + len(_validation_data["inputs"]) + len(_test_data["inputs"]))

Data type: <class 'dict'>
Total length: 46
Inputs Train Length 1107086
Inputs Validation Length 276770
Inputs Test Length 345962
Total Length:  1729818


In [8]:
m = ModelBuilder()
model = m._build_model()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, None, 46)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, None, 256)      │       310,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 46)             │         5,934 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 513,326 (1.96 MB)

 Trainable params: 513,326 (1.96 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train
history = model.fit(_train_data["inputs"], _train_data["targets"], 
                    validation_data=(_validation_data['inputs'], _validation_data["targets"]), 
                    batch_size=BATCH_SIZE, epochs=NUMBER_EPOCH)
# Evaluate
test_error, test_accuracy = model.evaluate(_test_data["inputs"], _test_data["targets"], verbose=1)
print("Accuracy on Test Set: ", test_accuracy)
print("Error on Test Set: ", test_error)